In [1]:
from tensorflow.keras.applications import Xception
import keras
from keras.layers import *
from keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = np.array([cv2.resize(img, (128, 128)) for img in X_train])
X_test = np.array([cv2.resize(img, (128, 128)) for img in X_test])

X_train = np.stack([X_train] * 3, axis=-1)
X_test = np.stack([X_test] * 3, axis=-1)

X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [3]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(128,128,3))
base_model.trainable = False

In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=outputs)

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
from keras.callbacks import EarlyStopping

batch_size = 32
epochs = 10

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience = 1,
    restore_best_weights = True
)

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping])

Epoch 1/10
1875/1875 [==============================] - 1195s 633ms/step - loss: 0.3280 - accuracy: 0.9093
Epoch 2/10
1875/1875 [==============================] - 1329s 709ms/step - loss: 0.1622 - accuracy: 0.9521
Epoch 3/10
1875/1875 [==============================] - 1231s 657ms/step - loss: 0.1297 - accuracy: 0.9610
Epoch 4/10
1875/1875 [==============================] - 1245s 664ms/step - loss: 0.1101 - accuracy: 0.9674
Epoch 5/10
1875/1875 [==============================] - 1189s 634ms/step - loss: 0.0987 - accuracy: 0.9707
Epoch 6/10
1875/1875 [==============================] - 1150s 613ms/step - loss: 0.0897 - accuracy: 0.9732
Epoch 7/10
1875/1875 [==============================] - 1102s 588ms/step - loss: 0.0814 - accuracy: 0.9760
Epoch 8/10
1875/1875 [==============================] - 1127s 601ms/step - loss: 0.0760 - accuracy: 0.9772
Epoch 9/10
1875/1875 [==============================] - 1182s 631ms/step - loss: 0.0711 - accuracy: 0.9786
Epoch 10/10
1875/1875 [==============

In [7]:
model.save('NN/model_mnist_xception.h5')

In [8]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_acc}')

313/313 [==============================] - 184s 582ms/step - loss: 0.1088 - accuracy: 0.9658

Test accuracy: 0.9657999873161316


In [9]:
from keras.saving.save import load_model

model = load_model('NN/model_mnist_xception.h5')

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_acc}')

313/313 [==============================] - 183s 581ms/step - loss: 0.1088 - accuracy: 0.9658

Test accuracy: 0.9657999873161316
